# Pre-Run Setup

### Import Libraries

In [1]:
from qiskit import *
import os
import datetime
import numpy as np
from datetime import date
#you have to load some account to start


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-_ixsll6g because the default path (/home/haimeng/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
import random
import os

In [3]:
# IBMQ.disable_account() 
# IBMQ.disable_account()
token0 = '316cca1ff42067b9901b6a9e90ca76496e76b4a73139954176c6fe31a45572fcd37089924cbe5c41edb96a9b4c0e92e44451ed350e2a7e45dc809d26cd752eed'
provider = IBMQ.enable_account(token0)

/home/haimeng/anaconda3/envs/newest_qiskit/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:109: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


### Specify the files that need to be run 
The old setup works with only one folder.

In [4]:
def shuffle_qasms(qasmlist):
    def get_time(filename):
        idgate = filename.split('/')[-1].split('_')[8].split('=')[-1]
        return idgate
    print(get_time(qasmlist[0]))
    times = list(set([get_time(x) for x in qasmlist])) #remove duplicates
    times.sort()
    print('# of time instances=%d'%(len(times)))
    random.shuffle(times) #randomize the timesteps
    print(times)
    sorted_qasm_list = []
    for time in times:
        sorted_qasm_list += [f for f in qasmlist if get_time(f) == time]
    return sorted_qasm_list

In [ ]:
import glob
from datetime import date
circuitPath = r"../../../Circuits"
device = "ibmq_bogota"
datestr = "20220228"
runtypes = [f.name for f in os.scandir(circuitPath + '/' + device + '/' + datestr) if f.is_dir() and 'QS4_Zplus' in f.name and 'Long' in f.name and 'NonMark' not in f.name and 'Spec' not in f.name]
batchFiles = []


TetraStates = ['tetra%d'%(i) for i in range(4)]
TetraStates.append('randomState')
runtype = [item for item in runtypes for i in range(5) if item.split('_')[2]==TetraStates[i] ]
runtypes_ordered = sorted(runtype,key=lambda t:t.split('_')[2],reverse=True)

In [ ]:
runtypes_ordered

In [ ]:
for runtype in runtypes_ordered:
    qasmpath = circuitPath + '/' + device + '/'+ datestr + '/' + runtype + '/'
    qasmlist = glob.glob(qasmpath + '%s*.qasm'%(runtype.split('_')[0]))
    # sort by id gates
    qasmlist.sort(key=lambda x: int(x.split('/')[-1].split('_')[8].split('=')[-1]))
    print(len(qasmlist))
    f=qasmlist[0]
    print(f)
    QuantumCircuit().from_qasm_file(f).draw(output='mpl')
    sorted_qasm_list = shuffle_qasms(qasmlist)
    # add measurement error mitigation
    meas_qasm = glob.glob(qasmpath + '%s*.qasm'%('MeasError'))
    sorted_qasm_list += meas_qasm
    print(len(sorted_qasm_list))
    batchFiles += [sorted_qasm_list]

In [ ]:
for i in range(5):
    print(batchFiles[i][0])

In [ ]:
backend = provider.backends.ibmq_bogota

In [ ]:
%pfile qiskit.transpile

In [ ]:
qasmlist[5]

In [ ]:
f=qasmlist[5]

print(f)
f = QuantumCircuit().from_qasm_file(f)
f = qiskit.transpile(f, backend = backend,  optimization_level=2)


In [ ]:
f.qasm()

In [ ]:
f.draw()

In [ ]:
for i in range(5):
    print(batchFiles[i][0].split('/')[-1].split('_')[2])

In [ ]:
tokens = {'haimeng@usc.edu':'316cca1ff42067b9901b6a9e90ca76496e76b4a73139954176c6fe31a45572fcd37089924cbe5c41edb96a9b4c0e92e44451ed350e2a7e45dc809d26cd752eed',
          'pokharel@usc.edu' : '21acf454868c89a86938ffcb6b17cc1236f783c6e340937ab3205598608408623893094770f403dd95ba758d9864ef2135eda8f381804787e15e8ab2aedd3ba3',
          'zhanghaimeng1994@gmail.com':'3513ac2a5100793f7e44bf520ad54b2d7bb68da03cd0f410a3bea14853644e1841de6ce86fbee2a750f814f032584d002f601fe25360b446f81339ba3a957eed',
          'theireasychair@gmail.com':'c59327488ff0575f9fcafd3f8d8ad627a1574fcf130508ac0314f08af78afc5cc5787acd7527140cb8aeacceaf4eaf82960785df5636991287d4f086db3b7229',
          'theireasychair@icloud.com':'e009883c3a103e728b741783f67d8586a37c0de04447151aa9493d13b3e76518f135424c482d9be2ef02759194085c8dce0db926f392ef5caa5b66f2843025a3',
          'jinweixu@stanford.edu':'21a7ae9539b78959e11dcc1383c5691767f36b13d5d8efa01e515b76714b796903c19a61537adf98583e1cb04e4ee2584c6ba5f632b9a16cb8ad80cc6ce90ba7'}

In [ ]:
apiDict = tokens
apiInv =dict(map(reversed, apiDict.items()))
apitokens = list(apiDict.values())

In [ ]:
apitokens

### Specify API and Backend

This script checks for how many jobs are available per token.

In [ ]:
#write the email address corresponding to each API, this will help when these APIs expire
apiDict = tokens
apiInv =dict(map(reversed, apiDict.items()))
apitokens = list(apiDict.values())


for token in apitokens:
    IBMQ.disable_account() #Start with no account loaded
    provider = IBMQ.enable_account(token) #Some account must be loaded before we start
    backend = provider.backends.ibmq_bogota
    available_jobs = backend.job_limit().maximum_jobs - backend.job_limit().active_jobs
    print(f"{available_jobs} jobs available for {apiInv[token]}")



In [ ]:
backend.configuration() #max_experiments=75, max_shots=8192
max_experiments=backend.configuration().max_experiments
max_shots=backend.configuration().max_shots
print(max_experiments,max_shots)

## print backend info

In [ ]:
prop=backend.properties()
type(prop)

In [ ]:
prop.qubit_property(1)

# Run the circuits

### Function to import backend and token information during the runs

In [ ]:
# print current working directory
print(os.getcwd())

In [ ]:
import csv
import os

runRecords = '../../../Data/records/info/' #information about each job is stored in this folder

def exportJobInfo(usertoken,jobid,filenames,backend):
    global runRecords
    now = datetime.datetime.now()
    qindex = int(filenames[0].split('/')[-1].split('_')[1][-1])
    with open(runRecords+jobid + '_' +usertoken+'.csv','w+') as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerow(['token',token])
        writer.writerow(['jobid',jobid])
        writer.writerow(['backend',backend.properties().backend_name])
        writer.writerow(['qubit',qindex])
        prop = backend.properties().qubit_property(qindex)
        for key,value in prop.items():
            writer.writerow([key,value])
        writer.writerow(['circuits'])
        for item in filenames:
            writer.writerow([item])
        # write time stamps
        job = backend.retrieve_job(runId)
        writer.writerow(['job_creation',job._time_per_step['CREATING']])
        

In [ ]:
backend_info = backend.properties().to_dict() 
backend_info['backend_name']

### Function to check for available tokens

In [ ]:

 
#Select a available token to send the run to
#With the current setup, loading and unloading a token takes sometime
#sequentially check each token for availability will increase loadtime for tokens that are near the end
#so given a list of tokens, everytime a job is sent to a token, that token will be sent to the end
def availableToken():
    global provider, backend, apitokens
    token = apitokens[0]
    available_jobs = backend.job_limit().maximum_jobs - backend.job_limit().active_jobs
    if available_jobs > 1:
        print(f'{available_jobs} jobs available for {apiInv[token]}')
        return token

    #if the first token has no availability then we will continue below
    apitokens.append(apitokens.pop(0))
    for token in apitokens:
        IBMQ.disable_account() 
        provider = IBMQ.enable_account(token) 
        backend = provider.backends.ibmq_bogota
        available_jobs = backend.job_limit().maximum_jobs - backend.job_limit().active_jobs
        if available_jobs > 1:
            print(f'{available_jobs} jobs available for {apiInv[token]}')
            apitokens.insert(0, apitokens.pop(apitokens.index(token)))
            return token

availableToken()   
      

### Actually doing the runs

In [ ]:
#start with empty containers
#dont run this block if you already have a list of runs you want to continue

runDict = {} #matches each batch to the job token
jobDict = {} #matches each batch to the entire job object
jobsFound = set([]) #status is updated when checked
jobsDone = set([])

In [ ]:
backend.properties().backend_name

In [ ]:
#start runs
#generally we want to run all the batchFiles, but we can also choose to run only some of them:
#todoFiles = [batchFiles[3]]
jobsNotFound = list(set(range(0, len(batchFiles))) - jobsFound)

# todoFiles = batchFiles[0::]
# for runFiles in todoFiles:
for n in jobsNotFound[0::]:
    
    runFiles = batchFiles[n]
    token = availableToken()

    allQasmList = [QuantumCircuit.from_qasm_file(file) for file in runFiles] #this needs to edited if the batch files also contain directory
    allCircuits= assemble(allQasmList, backend, shots=8192)

    job_current = backend.run(allCircuits)
    print(job_current.status())

    runId = job_current.job_id()
#     runTokens.append(token)
#     runIds.append(runId)
#         runDict[batchFiles.index(runFiles)] = [token, runId] #note that the index still refers to the original batchFiles
    runDict[n] = [token, runId]
    #as retrieve job is not working temporarily I will save job_current in a dictionary as well
#     jobDict[batchFiles.index(runFiles)] = job_current
    jobDict[n] = job_current
    print(f"Batch {batchFiles.index(runFiles)} has been sent")

    exportJobInfo(token, runId, batchFiles[0], backend)


print(runDict)

In [ ]:
runDict

### Checking job status

In [ ]:
# import sleep
jobsNotDone = list(set(runDict.keys()) - jobsDone)

def loadToken(token):
    global provider, backend
    if(not IBMQ.active_account()):
        IBMQ.load_account()
    IBMQ.disable_account() 
    provider = IBMQ.enable_account(token) 
    backend = provider.backends.ibmq_bogota
    return None

print("|Batch| Q#  | Status")
for job in jobsNotDone:
#     try:
    [token, job_id] = runDict[job]
    loadToken(token)

    #get job from dictionary instead of retrieve job as that function is broken for now
#         job_data = backend.retrieve_job(job_id)
    #alternatively get job from the jobD|ict
    if job in jobDict:
        job_data = jobDict[job]
    else:
        job_data = backend.retrieve_job(job_id)
    #print(f"Batch {job} is {job_data.status()}")

    jobsFound.add(job)
    qno = '---'
    if str(job_data.status()) == 'JobStatus.QUEUED':
        #print(f'queue number is {job_data.queue_info().position}')
        qno = str(job_data.queue_info().position).zfill(3)

    if str(job_data.status()) == 'JobStatus.DONE':
        jobsDone.add(job)

    st = str(job_data.status()).split('.')[1]
    jno = str(job).zfill(3)
    print(f'| {jno} | {qno} | {st}')
#     except:
#         pass

print(f'{len(jobsFound)} jobs found: {jobsFound}')
print(f'{len(jobsDone)} jobs done: {jobsDone}')
# if some batches are showing network errors, run only those batches again
# try, excess commands above force the loop to continue even when some batches show errors

# Exporting Data (Run after jobs have completed)

In [ ]:
filename = batchFiles[0][-1]
print(filename)
print(os.path.dirname(filename))
run=2


In [ ]:
filename

In [ ]:
resultRecords = '../../Data/records/results/'
if not os.path.exists(resultRecords):
        os.makedirs(resultRecords)

#job results are returned a list of dictionaries and they will have the same order as the order of the files in that batch
# filenames given here should also include the folder names as well
def exportBatchJob(filenames, token, job_id, index):
    loadToken(token)
    #job_data = backend.retrieve_job(job_id)
    if job in jobDict:
        job_data = jobDict[job]
        print('found token')
    else:
        job_data = backend.retrieve_job(job_id)
        
    # write finish time
    with open(runRecords+job_id + '_' +token+'.csv','a') as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerow(['job_finish_time',job_data._time_per_step['COMPLETED']])
        
    with open(resultRecords + f'{job_id}_{token}_results.txt', 'w') as f:
        f.write(str(job_data.result().to_dict()))
    
    for i in range(0, len(filenames)):
        result = job_data.result().get_counts()[i]
        exportResult(token, job_id, i, filenames[i], result,index)
        
        
#results will be in the form of dictionary
#filename here should also include the foldernames
def exportResult(token, job_id, circNo, filename, result,index):
#     print(filename)
    directory = os.path.dirname(filename).replace('/Circuits/', '/Data/raw/')+'_%d'%(index)+ f'/run{run}/'
    filename_new = os.path.basename(filename).split('.qasm')[0].split('_')
    
    if 'MeasMainqFree' in filename_new:
        filename_new.insert(5,str(index))
    file = directory + '_'.join(filename_new) + f'_{job_id}.txt'
    if not os.path.exists(os.path.dirname(file)):
        os.makedirs(os.path.dirname(file))
        
    
    with open(file, 'w') as f:
            f.write("usertoken,jobid,circuit_number\n")
            f.write(f"{token},{job_id},{circNo}\n")
            [f.write( '"'+str(key)+'"' + ',' + str(value) + '\n') for key, value in result.items()]
    return None




In [ ]:
# batchFilesFull = [ [ inputDir + f for f in batch] for batch in batchFiles] #this wont be necessary if the filename have dir
for job in list(runDict.keys()):
    [token, job_id] = runDict[job]
    #print(runDict[job])
    file = batchFiles[job][0]
    directory = os.path.dirname(file.replace('/Circuits/', '/Data/raw/'))
    if os.path.exists(directory):
        exists = os.listdir(directory)
        run = 1 + len(exists)
        print(run)
    else:
        run =2
    exportBatchJob(batchFiles[job], token, job_id,index=job)
    print(f"job {job} has been exported")